In [1]:
import pandas as pd
# glob module for wildcards in paths
import glob
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import math
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
start = time.time()
df = pd.read_csv(r"F:\WestbrookProject\SpaRe\control_group\pre-processing_2023\velocity_based\GazeProcessedMerge_correctedFixations_15_02_25.csv")
end = time.time()
print(end-start)
df

C:\Users\Vincent\AppData\Local\Temp\ipykernel_30052\2052686635.py:2: DtypeWarning: Columns (25,30,38,39,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"F:\WestbrookProject\SpaRe\control_group\pre-processing_2023\velocity_based\GazeProcessedMerge_correctedFixations_15_02_25.csv")


253.48561429977417


Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  timeStampRS  \
0                    0             0           0     0.000000   
1                    1             1           1     0.011111   
2                    2             2           2     0.022222   
3                    3             3           3     0.033333   
4                    4             4           4     0.044444   
...                ...           ...         ...          ...   
21066536      21066536      21066536    21066536   600.077778   
21066537      21066537      21066537    21066537   600.088889   
21066538      21066538      21066538    21066538   600.100000   
21066539      21066539      21066539    21066539   600.111111   
21066540      21066540      21066540    21066540   600.122222   

          eyeOpennessLeft  eyeOpennessRight  pupilDiameterMillimetersLeft  \
0                     1.0               1.0                      3.698715   
1                     1.0               1.0                      3.790924   
2                     1.0               1.0                      3.784473   
3                     1.0               1.0                      3.748617   
4                     1.0               1.0                      3.695748   
...                   ...               ...                           ...   
21066536              1.0               1.0                      3.024562   
21066537              1.0               1.0                      2.972967   
21066538              1.0               1.0                      2.962250   
21066539              1.0               1.0                      2.941080   
21066540              1.0               1.0                      2.902588   

          pupilDiameterMillimetersRight  eyePositionCombinedWorld_x  \
0                              4.148895                  -59.137527   
1                              4.124649                  -59.137662   
2                              4.124612                  -59.137514   
3                              4.121073                  -59.135959   
4                              4.107268                  -59.130572   
...                                 ...                         ...   
21066536                       2.846584                  144.161535   
21066537                       2.788579                  144.134964   
21066538                       2.790268                  144.122752   
21066539                       2.801670                  144.102621   
21066540                       2.822403                  144.089493   

          eyePositionCombinedWorld_y  ...  avg_dist  avg_dist_NH  \
0                           2.383395  ...       NaN          NaN   
1                           2.383442  ...  6.320999     6.320999   
2                           2.383491  ...  6.320999     6.320999   
3                           2.383605  ...  6.320999     6.320999   
4                           2.383912  ...  6.320999     6.320999   
...                              ...  ...       ...          ...   
21066536                   -2.669101  ...       NaN          NaN   
21066537                   -2.670622  ...       NaN          NaN   
21066538                   -2.671988  ...       NaN          NaN   
21066539                   -2.672585  ...       NaN          NaN   
21066540                   -2.672975  ...       NaN          NaN   

                   names         namesNH  long_events     SID  Session   ET  \
0                    NaN             NaN          NaN  1004.0      1.0  1.0   
1         pavement_O.002  pavement_O.002     0.481779  1004.0      1.0  1.0   
2         pavement_O.002  pavement_O.002     0.481779  1004.0      1.0  1.0   
3         pavement_O.002  pavement_O.002     0.481779  1004.0      1.0  1.0   
4         pavement_O.002  pavement_O.002     0.481779  1004.0      1.0  1.0   
...                  ...             ...          ...     ...      ...  ...   
21066536             NaN             NaN     0.224830  1080.0      5.0  3.0   
21

In [4]:
x_coords = df['eyePositionCombinedWorld_x']
y_coords = df['eyePositionCombinedWorld_y']
z_coords = df['eyePositionCombinedWorld_z']
df['coords'] = list(map(list, zip(x_coords, y_coords, z_coords)))
# zipping 3d coordinates together to calculate distances later
df['coords']

0           [-59.13752746582029, 2.38339519500732, 34.7818...
1           [-59.1376617904393, 2.38344215421863, 34.78179...
2           [-59.13751415122599, 2.38349143232834, 34.7817...
3           [-59.1359585573001, 2.3836050701958302, 34.781...
4           [-59.13057228136, 2.38391177165951, 34.7816720...
                                  ...                        
21066536    [144.161534652131, -2.66910078055657, -126.170...
21066537    [144.134964328742, -2.67062242639634, -126.209...
21066538    [144.12275150920098, -2.6719875153674906, -126...
21066539    [144.102620563918, -2.67258511742356, -126.260...
21066540    [144.089492797852, -2.6729748249054, -126.2723...
Name: coords, Length: 21066541, dtype: object

In [5]:
# 10 sec intervals
df['timeStampRS'] = pd.to_numeric(df['timeStampRS'])
df['timeStampRS']

closest_list = []
SIDs_n = []
Sessions_n = []
ETParts_n = []
ogIndex_n = []
distances = []
exact_timediffs = []
playerspeeds = []

for subject in np.unique(df.SID):
    df_subj = df[df['SID'] == subject]
    print("Starting subject:", subject)
    for session in np.unique(df_subj.Session):
        df_ses = df_subj[df_subj['Session'] == session]
        print("Starting session:", session)
        for part in np.unique(df_ses.ET):
            df_part = df_ses[df_ses['ET'] == part]
            ideal_times = list(range(0,round(float(df_part['timeStampRS'].iloc[-1])),10)) # for round down use math.floor(), else round() # 07.11. Adjusted to 10sec intervals
            real_times = [float(i) for i in list(df_part.timeStampRS)]
            for i in range(1,len(ideal_times)):
                closest_value = min(real_times, key=lambda x: abs(x - ideal_times[i])) 
                #print(f"The closest value to {target_values[i]} is {closest_value}")
                closest_list.append(closest_value)
                df_closest_value = df_part[df_part['timeStampRS'] == closest_value] ###
                prev_closest_value = min(real_times, key=lambda x: abs(x - ideal_times[i-1]))
                timediff = closest_value-prev_closest_value
                df_prev_closest_value = df_part[df_part['timeStampRS'] == prev_closest_value]
                #dist = math.dist(df_closest_value['coords'], df_prev_closest_value['coords'])
                coords = df_closest_value['coords'].iloc[0]
                prev_coords = df_prev_closest_value['coords'].iloc[0]
                dist = math.dist(coords, prev_coords)
                playerspeed = (dist / timediff) if timediff else np.nan # this marks all paused or interrupted recordings parts with NA-values (not 0)
                playerspeeds.append(playerspeed)
                exact_timediffs.append(timediff)
                distances.append(dist)
                ogIndex_n.append(df_closest_value['index'].iloc[0])
                SIDs_n.append(subject)
                Sessions_n.append(session)
                ETParts_n.append(part)
playerspeeds
print('These calculations are based on 10sec intervals:')
print('Maximum Speed:',max(playerspeeds), 'Minimum Speed:', min(playerspeeds), 'Average Speed (m/S):', np.nanmean(playerspeeds), 'Standard Deviation', np.nanstd(playerspeeds))

Starting subject: 1004.0
Starting session: 1.0
Starting session: 2.0
Starting session: 3.0
Starting session: 4.0
Starting session: 5.0
Starting subject: 1005.0
Starting session: 1.0
Starting session: 2.0
Starting session: 3.0
Starting session: 4.0
Starting session: 5.0
Starting subject: 1008.0
Starting session: 1.0
Starting session: 2.0
Starting session: 3.0
Starting session: 4.0
Starting session: 5.0
Starting subject: 1010.0
Starting session: 1.0
Starting session: 2.0
Starting session: 3.0
Starting session: 4.0
Starting session: 5.0
Starting subject: 1011.0
Starting session: 1.0
Starting session: 2.0
Starting session: 3.0
Starting session: 4.0
Starting session: 5.0
Starting subject: 1013.0
Starting session: 1.0
Starting session: 2.0
Starting session: 3.0
Starting session: 4.0
Starting session: 5.0
Starting subject: 1017.0
Starting session: 1.0
Starting session: 2.0
Starting session: 3.0
Starting session: 4.0
Starting session: 5.0
Starting subject: 1018.0
Starting session: 1.0
Starting

In [11]:
df_movement_10sec_timeframes = pd.DataFrame()
df_movement_10sec_timeframes['SID'] = SIDs_n
df_movement_10sec_timeframes['Session'] = Sessions_n
df_movement_10sec_timeframes['ET'] = ETParts_n
df_movement_10sec_timeframes['Mps'] = playerspeeds
df_movement_10sec_timeframes['Kmh'] = df_movement_10sec_timeframes['Mps']*3.6 
df_movement_10sec_timeframes['Distances'] = distances
df_movement_10sec_timeframes['TimeDiffs'] = exact_timediffs
df_movement_10sec_timeframes['Closest'] = closest_list
df_movement_10sec_timeframes['OGIndex'] = ogIndex_n

df_movement_10sec_timeframes

SID  Session   ET       Mps        Kmh  Distances  TimeDiffs  \
0      1004.0      1.0  1.0  0.006387   0.022992   0.063867       10.0   
1      1004.0      1.0  1.0  0.064635   0.232685   0.646347       10.0   
2      1004.0      1.0  1.0  0.030639   0.110302   0.306394       10.0   
3      1004.0      1.0  1.0  1.468286   5.285828  14.682857       10.0   
4      1004.0      1.0  1.0  1.341591   4.829728  13.415911       10.0   
...       ...      ...  ...       ...        ...        ...        ...   
23025  1080.0      5.0  3.0  3.277920  11.800513  32.779201       10.0   
23026  1080.0      5.0  3.0  2.851483  10.265340  28.514834       10.0   
23027  1080.0      5.0  3.0  2.765132   9.954477  27.651325       10.0   
23028  1080.0      5.0  3.0  4.481057  16.131805  44.810569       10.0   
23029  1080.0      5.0  3.0  3.851358  13.864889  38.513579       10.0   

       Closest   OGIndex  
0         10.0       900  
1         20.0      1800  
2         30.0      2700  
3         40.0      3600  
4         50.0      4500  
...        ...       ...  
23025    550.0  21062029  
23026    560.0  21062929  
23027    570.0  21063829  
23028    580.0  21064729  
23029    590.0  21065629  

[23030 rows x 9 columns]

In [21]:
# Meadian, SID & quantiles
SIDs = []
meds = []
p25s = []
p75s = []

for subject in np.unique(df_movement_10sec_timeframes.SID):
    df_subj = df_movement_10sec_timeframes[df_movement_10sec_timeframes['SID'] == subject]
    med = np.nanmedian(df_subj['Kmh'])
    p25 = np.nanquantile(df_subj['Kmh'], 0.25)
    p75 = np.nanquantile(df_subj['Kmh'], 0.75)
    SIDs.append(subject)
    meds.append(med)
    p25s.append(p25)
    p75s.append(p75)
    
df_speed = pd.DataFrame()
df_speed['SID'] = SIDs
df_speed['MoveSpeed_Kmh_25%'] = p25s
df_speed['MoveSpeed_Kmh_med'] = meds
df_speed['MoveSpeed_Kmh_75%'] = p75s
df_speed

SID  MoveSpeed_Kmh_25%  MoveSpeed_Kmh_med  MoveSpeed_Kmh_75%
0   1004.0           6.531851           9.846753          13.080364
1   1005.0           7.318599           9.458039          11.455881
2   1008.0          12.647020          15.814400          17.226562
3   1010.0          10.297154          13.919336          17.114279
4   1011.0           8.003310          10.737694          13.935457
5   1013.0          10.613084          13.739286          16.287415
6   1017.0           7.133411          10.092295          13.394869
7   1018.0           5.269136           9.811382          14.168060
8   1019.0          11.888877          14.871526          16.980153
9   1021.0           5.353452           8.323505          11.786688
10  1022.0           4.716666           6.362982           8.142112
11  1023.0          12.669914          15.367712          17.224704
12  1054.0           9.909235          13.301666          15.885281
13  1055.0           6.665700           9.015146          11.642195
14  1056.0          10.378020          15.016981          17.354403
15  1057.0           9.817313          12.992570          16.661897
16  1058.0           6.819420          10.708506          14.916090
17  1068.0           7.153609           9.058002          10.844063
18  1069.0          11.045374          14.418977          16.711956
19  1072.0          14.776627          16.896701          17.651890
20  1073.0          10.041337          13.082988          15.765372
21  1074.0           9.310106          13.471868          16.888725
22  1075.0          10.611587          13.879982          16.526789
23  1077.0           9.700839          13.345727          16.539136
24  1079.0           9.725667          13.820016          16.748473
25  1080.0           9.219952          12.890179          15.961570

In [23]:
# Calculate distance between 25th perceptile and 75th percentile & the Bowley coefficient to account for skew
df_speed['MoveSpeed_Kmh_DistQ1-Q3'] = df_speed['MoveSpeed_Kmh_75%'] - df_speed['MoveSpeed_Kmh_25%']
# BOWLEY COEFFICIENT: ranges from -1 to 1, with 0 indicating a symmetrical distribution, negative values indicating a left skew, and positive values indicating a right skew
df_speed['MoveSpeed_Kmh_bowleyCoef'] = (df_speed['MoveSpeed_Kmh_75%'] + df_speed['MoveSpeed_Kmh_25%'] - 2*df_speed['MoveSpeed_Kmh_med']) / df_speed['MoveSpeed_Kmh_DistQ1-Q3']

# drop columns not used in further analysis:
df_speed = df_speed.drop(['MoveSpeed_Kmh_25%', 'MoveSpeed_Kmh_75%'], axis=1)
df_speed

SID  MoveSpeed_Kmh_med  MoveSpeed_Kmh_DistQ1-Q3  \
0   1004.0           9.846753                 6.548513   
1   1005.0           9.458039                 4.137282   
2   1008.0          15.814400                 4.579542   
3   1010.0          13.919336                 6.817126   
4   1011.0          10.737694                 5.932147   
5   1013.0          13.739286                 5.674331   
6   1017.0          10.092295                 6.261458   
7   1018.0           9.811382                 8.898924   
8   1019.0          14.871526                 5.091276   
9   1021.0           8.323505                 6.433236   
10  1022.0           6.362982                 3.425445   
11  1023.0          15.367712                 4.554790   
12  1054.0          13.301666                 5.976045   
13  1055.0           9.015146                 4.976495   
14  1056.0          15.016981                 6.976383   
15  1057.0          12.992570                 6.844585   
16  1058.0          10.708506                 8.096670   
17  1068.0           9.058002                 3.690455   
18  1069.0          14.418977                 5.666581   
19  1072.0          16.896701                 2.875263   
20  1073.0          13.082988                 5.724035   
21  1074.0          13.471868                 7.578619   
22  1075.0          13.879982                 5.915202   
23  1077.0          13.345727                 6.838297   
24  1079.0          13.820016                 7.022805   
25  1080.0          12.890179                 6.741618   

    MoveSpeed_Kmh_bowleyCoef  
0                  -0.012414  
1                  -0.034225  
2                  -0.383274  
3                  -0.062671  
4                   0.078113  
5                  -0.101875  
6                   0.054890  
7                  -0.020853  
8                  -0.171670  
9                   0.076653  
10                  0.038773  
11                 -0.184598  
12                 -0.135343  
13                  0.055783  
14                 -0.329904  
15                  0.072184  
16                  0.039337  
17                 -0.032065  
18                 -0.190701  
19                 -0.474699  
20                 -0.062765  
21                 -0.098290  
22                 -0.105083  
23                 -0.066022  
24                 -0.166015  
25                 -0.088827

In [25]:
###SAVING###
df_speed.to_csv(r"C:\Users\Vincent\Documents\Dynamic_VS_Static_Viewing\FinalAnalysis\Preprocessing_ET\ResultingDataFrames\MovementSpeed_KmH_10sWindows.csv")